# Setup

Import required libraries including Tensorflow

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from keras.models import load_model

import random
import os
from numpy.random import seed

random.seed(42)
os.environ['PYTHONASHSEED'] = '42' 
seed(42)
tf.random.set_seed(42)

Check for Colab's GPU

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


Check connected GPU type

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jun  7 07:43:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    36W / 250W |    375MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Mount storage from Google Drive

In [4]:
from google.colab import drive
drive.mount('p2')

Drive already mounted at p2; to attempt to forcibly remount, call drive.mount("p2", force_remount=True).


# Dataset

In [5]:
df = pd.read_csv('/content/p2/MyDrive/p2/data/preprocessed_500k_imba.csv', dtype={'text': 'str', 'processed_text': 'str', 'stars': float})
df.fillna('', inplace=True)
df.head()

,text,stars,processed_text
0,Three words: Damn good pastries.\n\nA few mor...,4.0,three word damn good pastry word probably best...
1,Easily one of the worst Red Robin locations. T...,0.0,easily one worst red robin location food delic...
2,Maybe I am just spoiled with good Mexican food...,1.0,maybe spoiled good mexican food growing san di...
3,This Wildflower is always kept clean and the e...,4.0,wildflower always kept clean employee nice pot...
4,Favorite bibimbap in the valley! They also hav...,4.0,favorite bibimbap valley also korean fixing sm...


# Preprocessing

One-hot encoding of star labels

In [6]:
y = tf.keras.utils.to_categorical(df["stars"].values, num_classes=5)
y

array([[0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

Split dataset in stratified manner into train, validation and test set with proportion of 6:2:2

In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    df['processed_text'], 
    y, 
    test_size=0.2, 
    stratify=y, 
    random_state=42
)

x_train, x_val, y_train, y_val = train_test_split(
    x_train, 
    y_train, 
    test_size=0.25, 
    stratify=y_train, 
    random_state=42
)

print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(300000,)
(100000,)
(100000,)


Check the processed text and class

In [8]:
for i in range(5):
    print(x_train.iloc[i])
    print(y_train[i])
    
for i in range(5):
    print(x_val.iloc[i])
    print(y_val[i])

basically everything menu !had one thing !enjoy crab puppy best twist original hush puppy good vibe way around
[0. 0. 0. 0. 1.]
awesome middle eastern take hot sauce incredible love sandwich rock dish
[0. 0. 0. 0. 1.]
!rude completed order 10 minute lady drive arguing customer store literally yelled another mic !hear proceeded talk bad customer front 8 customer store !be back crew seems toxic wonder would act gm food beverage would expect far concerned update got new staff management gotten much better seems manager working hard customer service night day different glad see people striving better
[0. 0. 0. 1. 0.]
favorite dish tom kha shrimp soup order rice go along soup large enough serving meal pineapple fried rice chock full goody raisin cashew pineapple prefer shrimp meat addition tried chicken satay thai restaurant hand favorite archi would offer curry puff would !need go anywhere else best thai tea lunch special provide enough food although would like able choose soup come soup d

Tokenize the text with max vocabulary of 10,000

In [9]:
MAX_VOCAB_SIZE = 10000

tokenizer_10k = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer_10k.fit_on_texts(np.concatenate((x_train, x_val, x_test), axis=0))

x_train_10k = tokenizer_10k.texts_to_sequences(x_train)
x_val_10k = tokenizer_10k.texts_to_sequences(x_val)
x_test_10k = tokenizer_10k.texts_to_sequences(x_test)

vocab_size = len(tokenizer_10k.word_index) + 1  # Adding 1 because of reserved 0 index

for i in range(3):
  print(x_train_10k[i])

[759, 75, 21, 532, 9, 60, 237, 346, 2959, 23, 1821, 881, 3296, 2959, 3, 621, 67, 97]
[133, 743, 2130, 83, 108, 38, 691, 31, 81, 853, 44]
[502, 4670, 15, 215, 73, 497, 450, 4805, 120, 474, 585, 3673, 123, 7307, 856, 1935, 764, 99, 120, 325, 602, 120, 474, 196, 11, 2292, 431, 1421, 7, 1998, 3925, 1, 1249, 7, 374, 257, 2170, 1225, 20, 106, 42, 767, 991, 47, 59, 431, 220, 529, 252, 120, 6, 65, 80, 158, 524, 129, 54, 59]


Tokenize the text with max vocabulary of 1,000

In [10]:
MAX_VOCAB_SIZE = 1000

tokenizer_1k = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer_1k.fit_on_texts(np.concatenate((x_train, x_val, x_test), axis=0))

x_train_1k = tokenizer_1k.texts_to_sequences(x_train)
x_val_1k = tokenizer_1k.texts_to_sequences(x_val)
x_test_1k = tokenizer_1k.texts_to_sequences(x_test)

# vocab_size = len(tokenizer_1k.word_index) + 1  # Adding 1 because of reserved 0 index

for i in range(3):
  print(x_train_1k[i])

[759, 75, 21, 532, 9, 60, 237, 346, 23, 881, 3, 621, 67, 97]
[133, 743, 83, 108, 38, 691, 31, 81, 853, 44]
[502, 15, 215, 73, 497, 450, 120, 474, 585, 123, 856, 764, 99, 120, 325, 602, 120, 474, 196, 11, 431, 7, 1, 7, 374, 257, 20, 106, 42, 767, 991, 47, 59, 431, 220, 529, 252, 120, 6, 65, 80, 158, 524, 129, 54, 59]


Show the top 20 most frequent tokens

In [11]:
i = 0

for word in tokenizer_10k.word_index:
    print(f"{word} : {tokenizer_10k.word_index[word]}")
    i += 1
    if i >= 20:
        break

food : 1
place : 2
good : 3
great : 4
time : 5
service : 6
would : 7
like : 8
one : 9
get : 10
back : 11
go : 12
really : 13
restaurant : 14
order : 15
ordered : 16
u : 17
also : 18
chicken : 19
got : 20


The actual token size in the corpus

In [12]:
vocab_size

143134

Pad the tokens to fixed length of 100

In [13]:
max_length = 100

x_train_10k = tf.keras.preprocessing.sequence.pad_sequences(x_train_10k, padding='post', maxlen=max_length)
x_test_10k = tf.keras.preprocessing.sequence.pad_sequences(x_test_10k, padding='post', maxlen=max_length)
x_val_10k = tf.keras.preprocessing.sequence.pad_sequences(x_val_10k, padding='post', maxlen=max_length)

x_train_1k = tf.keras.preprocessing.sequence.pad_sequences(x_train_1k, padding='post', maxlen=max_length)
x_test_1k = tf.keras.preprocessing.sequence.pad_sequences(x_test_1k, padding='post', maxlen=max_length)
x_val_1k = tf.keras.preprocessing.sequence.pad_sequences(x_val_1k, padding='post', maxlen=max_length)

x_train_10k[:5, :]

array([[ 759,   75,   21,  532,    9,   60,  237,  346, 2959,   23, 1821,
         881, 3296, 2959,    3,  621,   67,   97,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0],
       [ 133,  743, 2130,   83,  108,   38,  691,   31,   81,  853,   44,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    

In [14]:
x_train_1k[:5, :]

array([[759,  75,  21, 532,   9,  60, 237, 346,  23, 881,   3, 621,  67,
         97,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0],
       [133, 743,  83, 108,  38, 691,  31,  81, 853,  44,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

Populate initial word embedding weights based on pre-trained GloVe

In [15]:
MAX_VOCAB_SIZE = 10000

embedding_dim = 100

def create_embedding_matrix(filepath, word_index, embedding_dim):
    embedding_matrix = np.zeros((MAX_VOCAB_SIZE, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index and (word_index[word] < MAX_VOCAB_SIZE):
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_matrix_10k = create_embedding_matrix(
    '/content/p2/MyDrive/p2/data/glove.6B.100d.txt',
    tokenizer_10k.word_index,
    embedding_dim
)

print(embedding_matrix_10k.shape)
embedding_matrix_10k[:5]

(10000, 100)


array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+0

In [16]:
MAX_VOCAB_SIZE = 1000

embedding_dim = 100

def create_embedding_matrix(filepath, word_index, embedding_dim):
    embedding_matrix = np.zeros((MAX_VOCAB_SIZE, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index and (word_index[word] < MAX_VOCAB_SIZE):
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_matrix_1k = create_embedding_matrix(
    '/content/p2/MyDrive/p2/data/glove.6B.100d.txt',
    tokenizer_1k.word_index,
    embedding_dim
)

print(embedding_matrix_1k.shape)
embedding_matrix_1k[:5]

(1000, 100)


array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+0

Calculate the percentage of vocabs found in GloVe and given initial weights

In [17]:
MAX_VOCAB_SIZE = 10000

nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix_10k, axis=1))
nonzero_elements / MAX_VOCAB_SIZE

0.9707

In [18]:
MAX_VOCAB_SIZE = 1000

nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix_1k, axis=1))
nonzero_elements / MAX_VOCAB_SIZE

0.999

# Modeling

Build CNN-LSTM model with 10,000 and 1,000 max vocab

In [19]:
from keras.regularizers import L1L2

MAX_VOCAB_SIZE = 10000

model_10k = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=MAX_VOCAB_SIZE,
        output_dim=embedding_dim,
        weights=[embedding_matrix_10k],
        input_length=max_length,
        trainable=True,
        mask_zero=True
    ),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(filters=100, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.LSTM(
        100, 
        recurrent_dropout=0.2,
        return_sequences=True
    ),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(100, activation='relu', kernel_regularizer='l2'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [20]:
from keras.regularizers import L1L2

MAX_VOCAB_SIZE = 1000

model_1k = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=MAX_VOCAB_SIZE,
        output_dim=embedding_dim,
        weights=[embedding_matrix_1k],
        input_length=max_length,
        trainable=True,
        mask_zero=True
    ),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(filters=100, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.LSTM(
        100, 
        recurrent_dropout=0.2,
        return_sequences=True
    ),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(100, activation='relu', kernel_regularizer='l2'),
    tf.keras.layers.Dense(5, activation='softmax')
])

Show summary of model

In [21]:
model_10k.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          1000000   
                                                                 
 dropout (Dropout)           (None, 100, 100)          0         
                                                                 
 conv1d (Conv1D)             (None, 100, 100)          30100     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 50, 100)          0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 50, 100)           80400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                        

In [22]:
model_1k.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 100)          100000    
                                                                 
 dropout_1 (Dropout)         (None, 100, 100)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 100, 100)          30100     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 50, 100)          0         
 1D)                                                             
                                                                 
 lstm_2 (LSTM)               (None, 50, 100)           80400     
                                                                 
 lstm_3 (LSTM)               (None, 100)               80400     
                                                      

Compile model with loss function and metrics

In [24]:
loss = tf.keras.losses.CategoricalCrossentropy()
metrics = tf.keras.metrics.CategoricalAccuracy(name='accuracy')

model_10k.compile(loss=loss,
              optimizer=tf.keras.optimizers.Adam(0.00083),
              metrics=metrics)

model_1k.compile(loss=loss,
              optimizer=tf.keras.optimizers.Adam(0.00083),
              metrics=metrics)

Train model

In [25]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=2)
checkpoint_filepath = '/content/p2/MyDrive/p2/cnn-lstm_10k-vocab/checkpoint'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

with tf.device('/device:GPU:0'):
  history = model_10k.fit(
      x_train_10k,
      y_train,
      epochs=8,
      validation_data=(x_val_10k, y_val),
      callbacks=[es, model_checkpoint_callback]
  )

Epoch 1/8
9375/9375 [==============================] - 1263s 134ms/step - loss: 0.9351 - accuracy: 0.6009 - val_loss: 0.8574 - val_accuracy: 0.6382
Epoch 2/8
9375/9375 [==============================] - 1183s 126ms/step - loss: 0.8147 - accuracy: 0.6482 - val_loss: 0.8090 - val_accuracy: 0.6531
Epoch 3/8
9375/9375 [==============================] - 1263s 135ms/step - loss: 0.7779 - accuracy: 0.6653 - val_loss: 0.7901 - val_accuracy: 0.6593
Epoch 4/8
9375/9375 [==============================] - 1264s 135ms/step - loss: 0.7498 - accuracy: 0.6780 - val_loss: 0.7891 - val_accuracy: 0.6617
Epoch 5/8
9375/9375 [==============================] - 1288s 137ms/step - loss: 0.7268 - accuracy: 0.6891 - val_loss: 0.7932 - val_accuracy: 0.6643
Epoch 6/8
9375/9375 [==============================] - 1272s 136ms/step - loss: 0.7065 - accuracy: 0.6994 - val_loss: 0.7903 - val_accuracy: 0.6653
Epoch 7/8
9375/9375 [==============================] - 1250s 133ms/step - loss: 0.6879 - accuracy: 0.7086 - val_

In [30]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=2)
checkpoint_filepath = '/content/p2/MyDrive/p2/cnn-lstm_1k-vocab/checkpoint'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

with tf.device('/device:GPU:0'):
  history = model_1k.fit(
      x_train_1k,
      y_train,
      epochs=8,
      validation_data=(x_val_1k, y_val),
      callbacks=[es, model_checkpoint_callback]
  )

Epoch 1/8
9375/9375 [==============================] - 1294s 138ms/step - loss: 0.9824 - accuracy: 0.6026 - val_loss: 0.9063 - val_accuracy: 0.6158
Epoch 2/8
9375/9375 [==============================] - 1291s 138ms/step - loss: 0.8692 - accuracy: 0.6269 - val_loss: 0.8526 - val_accuracy: 0.6339
Epoch 3/8
9375/9375 [==============================] - 1280s 137ms/step - loss: 0.8395 - accuracy: 0.6400 - val_loss: 0.8300 - val_accuracy: 0.6451
Epoch 4/8
9375/9375 [==============================] - 1227s 131ms/step - loss: 0.8200 - accuracy: 0.6472 - val_loss: 0.8265 - val_accuracy: 0.6471
Epoch 5/8
9375/9375 [==============================] - 1273s 136ms/step - loss: 0.8043 - accuracy: 0.6547 - val_loss: 0.8198 - val_accuracy: 0.6485
Epoch 6/8
9375/9375 [==============================] - 1288s 137ms/step - loss: 0.7916 - accuracy: 0.6611 - val_loss: 0.8145 - val_accuracy: 0.6532
Epoch 7/8
9375/9375 [==============================] - 1292s 138ms/step - loss: 0.7820 - accuracy: 0.6642 - val_

Evaluate model with val set

In [28]:
model_10k.load_weights(checkpoint_filepath)

y_pred_10k = model_10k.predict(x_val_10k, verbose=1)
y_pred_10k = np.argmax(y_pred_10k, axis=1)

print(classification_report(np.argmax(y_val, axis=1), y_pred_10k, digits=4))
confusion_matrix(np.argmax(y_val, axis=1), y_pred_10k)

3125/3125 [==============================] - 90s 29ms/step
              precision    recall  f1-score   support

           0     0.7252    0.7842    0.7536     11805
           1     0.5351    0.3881    0.4499      9287
           2     0.5359    0.4897    0.5117     13362
           3     0.5739    0.5434    0.5582     26144
           4     0.7562    0.8354    0.7938     39402

    accuracy                         0.6653    100000
   macro avg     0.6252    0.6082    0.6134    100000
weighted avg     0.6549    0.6653    0.6578    100000



array([[ 9258,  1521,   633,   171,   222],
       [ 2521,  3604,  2525,   463,   174],
       [  642,  1444,  6543,  4031,   702],
       [  157,   135,  2128, 14207,  9517],
       [  188,    31,   381,  5884, 32918]])

In [31]:
model_1k.load_weights(checkpoint_filepath)

y_pred_1k = model_1k.predict(x_val_1k, verbose=1)
y_pred_1k = np.argmax(y_pred_1k, axis=1)

print(classification_report(np.argmax(y_val, axis=1), y_pred_1k, digits=4))
confusion_matrix(np.argmax(y_val, axis=1), y_pred_1k)

3125/3125 [==============================] - 91s 29ms/step
              precision    recall  f1-score   support

           0     0.6881    0.7791    0.7308     11805
           1     0.4993    0.4267    0.4602      9287
           2     0.5467    0.4225    0.4766     13362
           3     0.5714    0.5053    0.5363     26144
           4     0.7359    0.8451    0.7867     39402

    accuracy                         0.6532    100000
   macro avg     0.6083    0.5957    0.5981    100000
weighted avg     0.6400    0.6532    0.6429    100000



array([[ 9197,  1646,   393,   191,   378],
       [ 2610,  3963,  1841,   524,   349],
       [  816,  1964,  5645,  3953,   984],
       [  330,   280,  2083, 13210, 10241],
       [  413,    84,   364,  5241, 33300]])

Evaluate model with test set

In [29]:
y_pred_10k = model_10k.predict(x_test_10k, verbose=1)
y_pred_10k = np.argmax(y_pred_10k, axis=1)

print(classification_report(np.argmax(y_test, axis=1), y_pred_10k, digits=4))
confusion_matrix(np.argmax(y_test, axis=1), y_pred_10k)

3125/3125 [==============================] - 88s 28ms/step
              precision    recall  f1-score   support

           0     0.7294    0.7943    0.7605     11805
           1     0.5304    0.3778    0.4413      9287
           2     0.5290    0.4844    0.5057     13362
           3     0.5684    0.5447    0.5563     26145
           4     0.7563    0.8301    0.7915     39401

    accuracy                         0.6630    100000
   macro avg     0.6227    0.6062    0.6110    100000
weighted avg     0.6527    0.6630    0.6556    100000



array([[ 9377,  1467,   617,   138,   206],
       [ 2502,  3509,  2612,   489,   175],
       [  616,  1469,  6472,  4054,   751],
       [  186,   142,  2171, 14240,  9406],
       [  175,    29,   362,  6130, 32705]])

In [32]:
y_pred_1k = model_1k.predict(x_test_1k, verbose=1)
y_pred_1k = np.argmax(y_pred_1k, axis=1)

print(classification_report(np.argmax(y_test, axis=1), y_pred_1k, digits=4))
confusion_matrix(np.argmax(y_test, axis=1), y_pred_1k)

3125/3125 [==============================] - 91s 29ms/step
              precision    recall  f1-score   support

           0     0.6854    0.7908    0.7343     11805
           1     0.4959    0.4055    0.4462      9287
           2     0.5366    0.4166    0.4691     13362
           3     0.5669    0.5082    0.5360     26145
           4     0.7374    0.8417    0.7861     39401

    accuracy                         0.6512    100000
   macro avg     0.6044    0.5926    0.5943    100000
weighted avg     0.6374    0.6512    0.6407    100000



array([[ 9335,  1586,   372,   161,   351],
       [ 2642,  3766,  1994,   540,   345],
       [  878,  1863,  5567,  4073,   981],
       [  350,   302,  2074, 13288, 10131],
       [  414,    78,   367,  5379, 33163]])